In [177]:
import re
import os
import shutil
import pandas as pd

# Borrar train folder

In [178]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [179]:
for di in os.listdir(path):
    shutil.rmtree(os.path.join(path, di, 'train'), ignore_errors=True)

# Borrar modelos malos

In [226]:
path = '/home/mr1142/Documents/Data/models/validation_results/validation_results' + '' + '.csv'
df = pd.read_csv(path)

In [227]:
df.sort_values('dice_coef_loss')

,type,name,loss,dice_coef_loss,loss_mask,accuracy,AUC,false_positives,false_negatives
17,uloss,renacimiento_validation_4,0.119882,0.030873,0.088655,0.975076,0.976777,80748.0,80958.0
15,uloss,renacimiento_validation_3,0.119526,0.030919,0.088200,0.975120,0.976994,80190.0,81231.0
12,unet,renacimiento_validation_2,0.032413,0.031955,0.089793,0.974418,0.973736,70983.0,94992.0
10,unet,renacimiento_validation_1,0.032621,0.032179,0.092485,0.973949,0.974434,88665.0,80353.0
14,unet,renacimiento_validation_3,0.033088,0.032720,0.098321,0.973591,0.974356,92355.0,78991.0
16,unet,renacimiento_validation_4,0.034823,0.034295,0.098386,0.972277,0.971930,88000.0,91869.0
2,unet,_albumentation_validation_2.h5,0.034487,0.034487,0.095812,0.973940,0.974166,7962.0,5701.0
7,uloss,albumentation_rotate_3_validation_2,0.129951,0.034909,0.094568,0.971946,0.973499,87304.0,94711.0
13,uloss,renacimiento_validation_2,0.131064,0.034971,0.095523,0.971893,0.973108,74191.0,108172.0
6,uloss,albumentation_rotate_3_validation_1,0.128454,0.035279,0.092766,0.971818,0.972174,78499.0,104346.0


In [194]:
malos_ = df[['type', 'name']][df.dice_coef_loss>0.04]

In [195]:
malos_

,type,name
12,uloss,retry_validation_rotate_1
13,uloss,retry_validation_gamma_1
14,unet,retry_validation_rotate_1
15,unet,retry_validation_gamma_1
18,uloss,retry_validation_rotate_2
19,unet,retry_validation_rotate_2
22,uloss,retry_validation_gamma_2
23,unet,retry_validation_gamma_2
25,uloss,retry_validation_rotate_3
27,unet,retry_validation_rotate_3


In [199]:
malos = [df['type'][i] + '_' + df['name'][i] for i in malos_.index]
malos = [re.sub('__', '_', mal) for mal in malos]
malos = [re.sub('.h5', '', mal) for mal in malos]

In [200]:
malos

['uloss_retry_validation_rotate_1',
 'uloss_retry_validation_gamma_1',
 'unet_retry_validation_rotate_1',
 'unet_retry_validation_gamma_1',
 'uloss_retry_validation_rotate_2',
 'unet_retry_validation_rotate_2',
 'uloss_retry_validation_gamma_2',
 'unet_retry_validation_gamma_2',
 'uloss_retry_validation_rotate_3',
 'unet_retry_validation_rotate_3',
 'uloss_retry_validation_gamma_3',
 'unet_retry_validation_gamma_3']

In [201]:
def match(lista_resultados, lista_match):
    return [f for f in lista_resultados if any([bool(re.search(match, f)) for match in lista_match])]

In [216]:
def eliminar(path, malos):
    files = os.listdir(path)
    files = match(files, malos)
    if len(files) == len(malos):
        for file in files:
            try:
                os.remove(os.path.join(path, file))
                print(f'file {file} removed')
            except:
                shutil.rmtree(os.path.join(path,file), ignore_errors=True)
                print(f'folder {file} removed')
    else:
        print('no tienen el mismo tamaño')
        print(files)

### Logs

In [217]:
path = '/home/mr1142/Documents/Data/logs/fit'

In [218]:
eliminar(path, malos)

folder unet_retry_validation_rotate_1 removed
folder uloss_retry_validation_rotate_1 removed
folder uloss_retry_validation_gamma_1 removed
folder unet_retry_validation_gamma_1 removed
folder uloss_retry_validation_rotate_2 removed
folder uloss_retry_validation_gamma_2 removed
folder unet_retry_validation_rotate_2 removed
folder uloss_retry_validation_rotate_3 removed
folder unet_retry_validation_gamma_2 removed
folder uloss_retry_validation_gamma_3 removed
folder unet_retry_validation_rotate_3 removed
folder unet_retry_validation_gamma_3 removed


### Modelos

In [219]:
path = '/home/mr1142/Documents/Data/models'

In [224]:
eliminar(path, malos)

file uloss_retry_validation_rotate_1.h5 removed
file uloss_retry_validation_gamma_1.h5 removed
file unet_retry_validation_rotate_1.h5 removed
file unet_retry_validation_gamma_1.h5 removed
file uloss_retry_validation_rotate_2.h5 removed
file unet_retry_validation_rotate_2.h5 removed
file uloss_retry_validation_gamma_2.h5 removed
file unet_retry_validation_gamma_2.h5 removed
file uloss_retry_validation_rotate_3.h5 removed
file unet_retry_validation_rotate_3.h5 removed
file uloss_retry_validation_gamma_3.h5 removed
file unet_retry_validation_gamma_3.h5 removed


# Borrar datos

In [225]:
path = '/home/mr1142/Documents/Data/models/validation_results'
csvs = os.listdir(path)

for csv in csvs:
    df = pd.read_csv(os.path.join(path, csv))
    index = [i for i in df.index if any([bool(re.search(mal, df['name'][i])) for mal in malos_['name']])]
    df = df.drop(index)
    df.to_csv(os.path.join(path, csv), index = False)

False